In [1]:
%run clean_data_en_lstm
%run clean_features

In [2]:
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import duolingo_replica_alex as dr
import duolingo_replica as d
import dask.dataframe as dd

In [3]:
file_path = "C:/Users/Alexander/Dropbox/halflife_regression_rl/0_data/learning_traces.13m.csv"
result_df_en_top20 = process_data(file_path, 20)
result_df_en_top20

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct,appearance_count
0,1.0,1362082504,357,u:dwbJ,en,pt,1052c3ace653dbc8923eaa183bc02b88,definition/definition<n><sg>,17,17,2,2,629
1,1.0,1362082504,357,u:dwbJ,en,pt,9cba1b30f88bf3c047b22cffcaf88c12,surface/surface<n><sg>,19,19,3,3,629
2,1.0,1362082504,357,u:dwbJ,en,pt,961cd149f20f2571419b1412d849f19a,scale/scale<n><sg>,21,20,3,3,629
3,0.8,1362082504,357,u:dwbJ,en,pt,5cbb1249562e95794a4c4ae0e2d8ae26,temperature/temperature<n><sg>,44,36,5,4,629
4,1.0,1362082504,357,u:dwbJ,en,pt,2df65bdf80d10d2b78d62cb2e0a731d8,distance/distance<n><sg>,21,20,3,3,629
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569115,1.0,1363104881,4294673,u:f_W4,en,pt,c032182c0ffb744c963ec83c937a26f4,us/prpers<prn><obj><p1><mf><pl>,10,9,1,1,35
569116,1.0,1363104881,4666611,u:f_W4,en,pt,80765ae2a08ba6812a4ab9df607b97d2,am/be<vbser><pri><p1><sg>,21,19,1,1,35
569117,0.5,1363104881,3543733,u:f_W4,en,pt,6d4c572af8022cb4784ce0f8898d1905,to/to<pr>,4,4,2,1,35
569118,1.0,1363104881,3610474,u:f_W4,en,pt,c9fb923e49d5cba24b5afb9ee1cff2a9,wine/wine<n><sg>,21,18,1,1,35


In [4]:
clean_data, feature_vars = read_data( result_df_en_top20, omit_lexemes = False )

In [5]:
clean_data

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,en:your/your<det><pos><sp>,en:yours/yours<prn><pos><mf><sp>,en:yourself/yourself<prn><ref><p2><mf><sg>,en:youth/youth<n><sg>,en:zone/zone<n><sg>,time_of_day,time_afternoon,time_evening,time_morning,time_night
0,1.0,1362082504,357,u:dwbJ,en,pt,1052c3ace653dbc8923eaa183bc02b88,definition/definition<n><sg>,17,17,...,0.0,0.0,0.0,0.0,0.0,evening,0,1,0,0
1,1.0,1362082504,357,u:dwbJ,en,pt,9cba1b30f88bf3c047b22cffcaf88c12,surface/surface<n><sg>,19,19,...,0.0,0.0,0.0,0.0,0.0,evening,0,1,0,0
2,1.0,1362082504,357,u:dwbJ,en,pt,961cd149f20f2571419b1412d849f19a,scale/scale<n><sg>,21,20,...,0.0,0.0,0.0,0.0,0.0,evening,0,1,0,0
3,0.8,1362082504,357,u:dwbJ,en,pt,5cbb1249562e95794a4c4ae0e2d8ae26,temperature/temperature<n><sg>,44,36,...,0.0,0.0,0.0,0.0,0.0,evening,0,1,0,0
4,1.0,1362082504,357,u:dwbJ,en,pt,2df65bdf80d10d2b78d62cb2e0a731d8,distance/distance<n><sg>,21,20,...,0.0,0.0,0.0,0.0,0.0,evening,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569115,1.0,1363104881,4294673,u:f_W4,en,pt,c032182c0ffb744c963ec83c937a26f4,us/prpers<prn><obj><p1><mf><pl>,10,9,...,0.0,0.0,0.0,0.0,0.0,afternoon,1,0,0,0
569116,1.0,1363104881,4666611,u:f_W4,en,pt,80765ae2a08ba6812a4ab9df607b97d2,am/be<vbser><pri><p1><sg>,21,19,...,0.0,0.0,0.0,0.0,0.0,afternoon,1,0,0,0
569117,0.5,1363104881,3543733,u:f_W4,en,pt,6d4c572af8022cb4784ce0f8898d1905,to/to<pr>,4,4,...,0.0,0.0,0.0,0.0,0.0,afternoon,1,0,0,0
569118,1.0,1363104881,3610474,u:f_W4,en,pt,c9fb923e49d5cba24b5afb9ee1cff2a9,wine/wine<n><sg>,21,18,...,0.0,0.0,0.0,0.0,0.0,afternoon,1,0,0,0


In [6]:
feature_vars_time = feature_vars + ['time_afternoon', 'time_evening', 'time_morning', 'time_night']
# print(feature_vars_time)

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming clean_data is your DataFrame, Y is the target column, and feature_vars are the features
# Extract unique users
unique_users = clean_data['user_id'].unique()  # Replace 'user_id' with your user identifier column

# Split users into train and test groups
train_users, test_users = train_test_split(unique_users, test_size=0.2, random_state=42)  # Adjust test_size as needed

# Create training and testing datasets
train_data = clean_data[clean_data['user_id'].isin(train_users)]
test_data = clean_data[clean_data['user_id'].isin(test_users)]

In [8]:
# Pre-sort the data by 'user_id' and 'datetime'
train_data.sort_values(by=['user_id', 'datetime'], inplace= True)
test_data.sort_values(by=['user_id', 'datetime'], inplace= True)

c:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
train_data.user_id.nunique()
train_data.shape

(455280, 2325)

## Create Batches so we cann run the model

In [10]:
import numpy as np

def create_non_overlapping_sequences(data, feature_vars, time_steps=20, batch_size=1000):
    # Convert feature columns to numpy for faster processing
    feature_data = data[feature_vars].to_numpy()
    output_data = data['p'].to_numpy()  # Assuming 'p' is your target variable

    # Get the starting index for each new user
    user_change_indices = np.where(data['user_id'].to_numpy()[:-1] != data['user_id'].to_numpy()[1:])[0] + 1
    user_start_indices = np.insert(user_change_indices, 0, 0)

    for batch_start in range(0, len(user_start_indices), batch_size):
        sequences = []
        outputs = []

        # Iterate through each user in the batch
        for i in range(batch_start, min(batch_start + batch_size, len(user_start_indices))):
            start_idx = user_start_indices[i]
            end_idx = start_idx + time_steps if i + 1 < len(user_start_indices) else len(feature_data)

            # Check if the user data is exactly equal to time_steps
            if end_idx - start_idx == time_steps:
                sequences.append(feature_data[start_idx:end_idx])
                outputs.append(output_data[end_idx - 1])  # Target for the last period in the sequence

        yield np.array(sequences), np.array(outputs)


# Complex Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

model = Sequential()
model.add(LSTM(100, input_shape=(20, len(feature_vars_time)), return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu', return_sequences=False))
model.add(Dense(1, kernel_regularizer=L1L2(l1=0.01, l2=0.01)))
model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [18]:
from tensorflow.keras.metrics import MeanAbsoluteError
import numpy as np

# Initialize the MAE metric
mae_metric = MeanAbsoluteError()
num_epochs = 10
# Train the model using the generator
for epoch in range(num_epochs):  # You can define the number of epochs
    print("Epoch {}/{}".format(epoch + 1, num_epochs))
    for X_batch, Y_batch in create_non_overlapping_sequences(train_data, feature_vars_time):
        # Train on batch and compute loss
        loss = model.train_on_batch(X_batch, Y_batch)

        # Compute MAE
        predictions = model.predict_on_batch(X_batch)
        mae_metric.update_state(Y_batch, predictions)
        mae = mae_metric.result().numpy()  # Get the MAE value

        # Reset MAE metric at the end of each batch
        mae_metric.reset_states()

        # Optionally print the loss and MAE for each batch
        print("Batch Loss: {:.4f}, Batch MAE: {:.4f}".format(loss, mae))

Epoch 1/10
Batch Loss: 0.0752, Batch MAE: 0.1361
Batch Loss: 0.0832, Batch MAE: 0.1623
Batch Loss: 0.0762, Batch MAE: 0.1864
Batch Loss: 0.0876, Batch MAE: 0.2091
Batch Loss: 0.0797, Batch MAE: 0.1945
Batch Loss: 0.0917, Batch MAE: 0.1880
Batch Loss: 0.0764, Batch MAE: 0.1484
Batch Loss: 0.0712, Batch MAE: 0.1301
Batch Loss: 0.0859, Batch MAE: 0.1581
Batch Loss: 0.0806, Batch MAE: 0.1773
Batch Loss: 0.0805, Batch MAE: 0.2041
Batch Loss: 0.0733, Batch MAE: 0.1988
Batch Loss: 0.0801, Batch MAE: 0.1885
Batch Loss: 0.0695, Batch MAE: 0.1519
Batch Loss: 0.0587, Batch MAE: 0.1233
Batch Loss: 0.0562, Batch MAE: 0.1137
Batch Loss: 0.0751, Batch MAE: 0.1509
Batch Loss: 0.0593, Batch MAE: 0.1660
Batch Loss: 0.0718, Batch MAE: 0.1943
Batch Loss: 0.0706, Batch MAE: 0.1824
Batch Loss: 0.0730, Batch MAE: 0.1704
Batch Loss: 0.0565, Batch MAE: 0.1317
Batch Loss: 0.0697, Batch MAE: 0.1407
Epoch 2/10
Batch Loss: 0.0725, Batch MAE: 0.1361
Batch Loss: 0.0807, Batch MAE: 0.1617
Batch Loss: 0.0749, Batch MA

In [15]:
# Initialize metric for testing
test_mae_metric = MeanAbsoluteError()

# Run model on test data
for X_test_batch, Y_test_batch in create_non_overlapping_sequences(test_data, feature_vars_time):
    # Make predictions
    test_predictions = model.predict_on_batch(X_test_batch)

    # Update MAE metric
    test_mae_metric.update_state(Y_test_batch, test_predictions)
    print(test_mae_metric.result())

# Calculate final MAE on test data
final_test_mae = test_mae_metric.result().numpy()
print("Final Test MAE: {:.6f}".format(final_test_mae))

tf.Tensor(0.16631444, shape=(), dtype=float32)
tf.Tensor(0.16595434, shape=(), dtype=float32)
tf.Tensor(0.16862732, shape=(), dtype=float32)
tf.Tensor(0.1678477, shape=(), dtype=float32)
tf.Tensor(0.16855097, shape=(), dtype=float32)
tf.Tensor(0.16897923, shape=(), dtype=float32)
Final Test MAE: 0.168979
